In [7]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os.path as osp

# Device configuration 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
# Hyper parameters 
num_epochs = 5
num_classes = 10
batch_size = 8
learning_rate = 0.001

In [9]:
# MNIST dataset 
path = './data/'

train_dataset = torchvision.datasets.MNIST(root=path,
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root=path,
                                          train=False, 
                                          transform=transforms.ToTensor()) 

100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:00<00:00, 27.5MB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 3.71MB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 13.5MB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<?, ?B/s]


In [10]:
# Data loader 设置数据集加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [11]:
# Convolutional neural network (two convolutional layers) 
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)
model

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [12]:
# Loss and optimizer 
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# Train the model 
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/7500], Loss: 0.1459
Epoch [1/5], Step [200/7500], Loss: 0.0894
Epoch [1/5], Step [300/7500], Loss: 1.0016
Epoch [1/5], Step [400/7500], Loss: 0.0110
Epoch [1/5], Step [500/7500], Loss: 0.0775
Epoch [1/5], Step [600/7500], Loss: 0.0931
Epoch [1/5], Step [700/7500], Loss: 0.5481
Epoch [1/5], Step [800/7500], Loss: 0.0395
Epoch [1/5], Step [900/7500], Loss: 0.0127
Epoch [1/5], Step [1000/7500], Loss: 0.0010
Epoch [1/5], Step [1100/7500], Loss: 0.0213
Epoch [1/5], Step [1200/7500], Loss: 0.3254
Epoch [1/5], Step [1300/7500], Loss: 0.0131
Epoch [1/5], Step [1400/7500], Loss: 0.0398
Epoch [1/5], Step [1500/7500], Loss: 0.0019
Epoch [1/5], Step [1600/7500], Loss: 0.0223
Epoch [1/5], Step [1700/7500], Loss: 0.0068
Epoch [1/5], Step [1800/7500], Loss: 0.0357
Epoch [1/5], Step [1900/7500], Loss: 0.0155
Epoch [1/5], Step [2000/7500], Loss: 0.0598
Epoch [1/5], Step [2100/7500], Loss: 0.0095
Epoch [1/5], Step [2200/7500], Loss: 0.0511
Epoch [1/5], Step [2300/7500], Loss: 0.06

In [14]:
# Test the model 
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.99 %
